In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import OrderedDict
from functools import partial

from tqdm.contrib.concurrent import process_map
# from multiprocessing import Pool

import aoc
from rangemap import RangeMap

TypeError: jitclass() got an unexpected keyword argument 'nopython'

In [4]:
input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [5]:
almanac = lambda input: OrderedDict({
    m.split('map:\n')[0].strip(): RangeMap(m.split('map:\n')[1])
    for m in input.strip().split('\n\n')
    if not m.startswith('seeds:')
})
almanac(input)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1mCannot resolve setattr: (instance.jitclass.RangeMap#24c90363580<str_repr:unicode_type>).source_ranges = list(undefined)<iv=None>
[1m
File "rangemap.py", line 14:[0m
[1m    def __init__(self, str_repr):
[1m        self.source_ranges = []
[0m        [1m^[0m[0m
[0m
[0m[1mDuring: typing of set attribute 'source_ranges' at c:\Users\yelkh\Documents\aoc\2023\5\rangemap.py (14)[0m
[1m
File "rangemap.py", line 14:[0m
[1m    def __init__(self, str_repr):
[1m        self.source_ranges = []
[0m        [1m^[0m[0m

[0m[1mDuring: resolving callee type: jitclass.RangeMap#24c90363580<str_repr:unicode_type>[0m
[0m[1mDuring: typing of call at <string> (3)
[0m
[0m[1mDuring: resolving callee type: jitclass.RangeMap#24c90363580<str_repr:unicode_type>[0m
[0m[1mDuring: typing of call at <string> (3)
[0m
[1m
File "<string>", 

In [106]:
def seed_to(seed, dest_map_name, almanac, debug=False):
    index = seed
    if debug: print(f"seed {seed}", end=" ")
    for key, correspondance in almanac.items():
        index = correspondance[index]
        if debug: print(f"-> {key.split('-')[-1]} {index}", end=" ")
        if key == dest_map_name:
            return index
    return index

In [108]:
%%timeit

seed_to(79, "humidity-to-location", almanac(input), debug=False)

39.4 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [101]:
%%timeit

seed_to(79, "humidity-to-location", almanac(input), debug=False)

38.4 µs ± 353 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [100]:
seed_to(79, "humidity-to-location", almanac(input), debug=True)

seed 79 -> soil 81 -> fertilizer 81 -> water 81 -> light 74 -> temperature 78 -> humidity 78 -> location 82 

82

In [86]:
seed_to(79, "humidity-to-location", almanac(input), debug=False)

82

In [69]:
def seed_to_location(seed, almanac):
    return seed_to(seed, "humidity-to-location", almanac)

In [70]:
seed_to_location(48, almanac(input))

65

In [71]:
seeds = lambda input: list(map(int, input.strip().split('\n')[0].split(':')[1].strip().split()))
seeds(input)

[79, 14, 55, 13]

In [64]:
almanac(input)

OrderedDict([('seed-to-soil',
              (0, 85096661) -> (84834756, 169931417)
              (85096662, 159718197) -> (0, 74621535)
              (159718198, 169931417) -> (74621536, 84834755)
              (169931418, 286288422) -> (1281808941, 1398165945)
              (286288423, 327434958) -> (1398165946, 1439312481)
              (327434959, 922461290) -> (636682580, 1231708911)
              (922461291, 960074060) -> (1244196171, 1281808940)
              (960074061, 1078390709) -> (1439312482, 1557629130)
              (1078390710, 1432361567) -> (169931418, 523902275)
              (1432361568, 1436236333) -> (523902276, 527777041)
              (1436236334, 1448723592) -> (1231708912, 1244196170)
              (1448723593, 1557629130) -> (527777042, 636682579)
              (1621489177, 1746859317) -> (3966424878, 4091795018)
              (1746859318, 1814797941) -> (1621489177, 1689427800)
              (1814797942, 1867878055) -> (3495880502, 3548960615)
              (

In [57]:
list(zip(seeds(input)[::2], seeds(input)[1::2]))

[(1187290020, 247767461),
 (40283135, 64738286),
 (2044483296, 66221787),
 (1777809491, 103070898),
 (108732160, 261552692),
 (3810626561, 257826205),
 (3045614911, 65672948),
 (744199732, 300163578),
 (3438684365, 82800966),
 (2808575117, 229295075)]

In [88]:
def closest_location(seeds, almanac):
    return min([seed_to_location(seed, almanac) for seed in seeds])

input = aoc.input(5)
answer = closest_location(seeds(input), almanac(input))
answer

282277027

In [93]:
%%timeit

closest_location(seeds(input), almanac(input))

1.19 ms ± 18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [91]:
%%timeit

closest_location(seeds(input), almanac(input))

827 µs ± 15.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [17]:
input = aoc.input(5)
answer = closest_location((1187290020, 247767461), almanac(input))
answer

C:\Users\yelkh\AppData\Local\Temp\ipykernel_20752\1083351717.py:2: TqdmWarning: Iterable length 247767461 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  result = process_map(
